In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE RecordWildCards #-}

import AdventOfCode

import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Map.Strict as M
import qualified Data.Vector as V

input <- dayLines 18

In [2]:
type Reg = Char
type ValueReg = Either Int Reg

parseReg = A.satisfy A.isAlpha_ascii
parseValue = A.signed A.decimal

parseValueReg :: A.Parser ValueReg
parseValueReg = A.eitherP parseValue parseReg

In [3]:
data Instr 
    = Snd ValueReg 
    | Set Reg ValueReg
    | Add Reg ValueReg
    | Mul Reg ValueReg
    | Mod Reg ValueReg
    | Rcv Reg
    | Jgz ValueReg ValueReg
    deriving (Eq, Show)

parseInstr = A.choice
    [ Snd <$> ("snd" *> A.space *> parseValueReg)
    , Set <$> ("set" *> A.space *> parseReg)      <*> (A.space *> parseValueReg)
    , Add <$> ("add" *> A.space *> parseReg)      <*> (A.space *> parseValueReg)
    , Mul <$> ("mul" *> A.space *> parseReg)      <*> (A.space *> parseValueReg)
    , Mod <$> ("mod" *> A.space *> parseReg)      <*> (A.space *> parseValueReg)
    , Rcv <$> ("rcv" *> A.space *> parseReg)
    , Jgz <$> ("jgz" *> A.space *> parseValueReg) <*> (A.space *> parseValueReg)
    ]

registerMap = M.fromList $ zip ['a'..'p'] (repeat 0)
instrVec    = V.fromList $ map (parsed parseInstr) input

In [4]:
data MachineState = MachineState
    { registers :: M.Map Char Int
    , counter   :: Int
    } deriving (Eq, Show)

getValueReg registers valueReg = case valueReg of
    Left v -> v
    Right r -> registers M.! r

executeInstr instr ms@MachineState{..} = case instr of
    Snd     vr -> ms{counter = counter + 1}
    Set reg vr -> ms{registers = M.insert reg    (getValueReg registers vr)      registers, counter = counter + 1}
    Add reg vr -> ms{registers = M.adjust      (+(getValueReg registers vr)) reg registers, counter = counter + 1}
    Mul reg vr -> ms{registers = M.adjust      (*(getValueReg registers vr)) reg registers, counter = counter + 1}
    Mod reg vr -> ms{registers = M.adjust (`mod` (getValueReg registers vr)) reg registers, counter = counter + 1}
    Jgz vr vr' -> if (getValueReg registers vr) > 0
        then ms{counter = counter + (getValueReg registers vr')}
        else ms{counter = counter + 1}
    other -> error $ "don't know how to execute " ++ show other

In [5]:
loop instrs ms@MachineState{..} lastFreq | counter > V.length instrs || counter < 0 = lastFreq
loop instrs ms@MachineState{..} lastFreq = case instrs V.! counter of
    Snd vr -> loop instrs (executeInstr (Snd vr) ms) (getValueReg registers vr)
    Rcv r  -> if (registers M.! r) /= 0 then lastFreq else loop instrs ms{counter = counter + 1} lastFreq
    curr   -> loop instrs (executeInstr curr ms) lastFreq

In [6]:
loop instrVec (MachineState registerMap 0) 0

3423

In [7]:
send msg cms@ConcMachineState{..} = cms{mailbox = mailbox ++ [msg]}

data LockedState = Running | Waiting deriving (Eq, Show)
data ConcMachineState = ConcMachineState
    { ms          :: MachineState
    , mailbox     :: [Int]
    , lockedState :: LockedState
    }

executeInstr' instr cms@ConcMachineState{..} = case instr of
    Rcv reg -> if mailbox == [] 
        then cms{lockedState = Waiting}
        else let
            (msg, mailbox') = (head mailbox, tail mailbox)
            registers' = M.insert reg msg $ registers ms
            in cms{ms = ms{registers = registers', counter = counter ms + 1}, mailbox = mailbox', lockedState = Running}
    _ -> cms{ms = executeInstr instr ms}

loop' instrs cms0 cms1 count
    | (lockedState cms0) == Waiting && (lockedState cms1) == Waiting = count
    | (counter $ ms cms0) > V.length instrs || (counter $ ms cms0) < 0 = count
    | (counter $ ms cms1) > V.length instrs || (counter $ ms cms1) < 0 = count
    | Snd vr <- instrs V.! (counter $ ms cms0) = let
        value = getValueReg (registers $ ms cms0) vr
        cms1' = send value cms1
        cms0' = executeInstr' (Snd vr) cms0
        in loop' instrs cms0' cms1' count
    | Snd vr <- instrs V.! (counter $ ms cms1) = let
        value = getValueReg (registers $ ms cms1) vr
        cms0' = send value cms0
        cms1' = executeInstr' (Snd vr) cms1
        in loop' instrs cms0' cms1' (count + 1)
    | otherwise = let
        instr0 = instrs V.! (counter $ ms cms0)
        instr1 = instrs V.! (counter $ ms cms1)
        cms0' = executeInstr' instr0 cms0
        cms1' = executeInstr' instr1 cms1
        in loop' instrs cms0' cms1' count

initialMs0 = ConcMachineState (MachineState                 registerMap  0) [] Running
initialMs1 = ConcMachineState (MachineState (M.insert 'p' 1 registerMap) 0) [] Running

loop' instrVec initialMs0 initialMs1 0

7493